In [13]:
import os
import glob
import pandas as pd
from tqdm import tqdm
import json
from transformers import WhisperForConditionalGeneration, WhisperProcessor, BitsAndBytesConfig
import torch
from utils.clean_transcript import clean
from peft import PeftModel
import librosa
from IPython.display import Audio, display

In [14]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # available GPU
lang = 'bew' # which language to see
num_examples = 5 # how many sentences to transcribe/view/play

In [15]:
def generate(model, data, processor, proxy_lang):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_dtype = next(model.parameters()).dtype
    model.to(device).eval()
    forced_decoder_ids = processor.get_decoder_prompt_ids(language=proxy_lang, task="transcribe")
    predictions = []
    filepaths = []
    with torch.no_grad():
        for filepath in tqdm(data):      
            audio = librosa.load(filepath, offset=0, duration=30, mono=True, sr=16_000)[0]
            inputs = processor(audio=[audio], sampling_rate=16_000, return_tensors='pt')
            input_features = inputs.input_features.to(model.device)
            input_features = input_features.to(dtype=input_dtype)      
            
           # Generate output token IDs
            predicted_ids = model.generate(
                input_features,
                forced_decoder_ids=forced_decoder_ids,
                max_new_tokens=200
            )
            transcriptions = processor.batch_decode(predicted_ids, skip_special_tokens=True)
            predictions += transcriptions
            filepaths.append(filepath)
    model.to("cpu")
    return predictions, filepaths

In [16]:
def get_model(config, model_dir, lang):
    if config['lora']:  
        # quantize
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        model = WhisperForConditionalGeneration.from_pretrained(
            config["whisper_model"],
            quantization_config=bnb_config
        )
        model = PeftModel.from_pretrained(model, f"{model_dir}/{lang}")        
        model.print_trainable_parameters()
    else:
        model = WhisperForConditionalGeneration.from_pretrained(f"{model_dir}/{lang}")
    return model 

In [17]:
with open("config.json", "r") as f:
    config = json.load(f)
    f.close()

In [18]:
audio_paths = f"/cs-data-01/drd92/mozilla-asr-challenge/mcv-sps-st-09-2025/sps-corpus-1.0-2025-09-05-{lang}/audios/*"
model_dir = "results/whisper-large-v3/final_models"

In [19]:
proxy_lang = config["proxy_langs"][lang]
processor = WhisperProcessor.from_pretrained(
    config["whisper_model"],
    language=proxy_lang,
    task="transcribe"
)

In [20]:
data = glob.glob(audio_paths)[:num_examples]

In [21]:
model = get_model(config, model_dir, lang)

In [22]:
predictions, filepaths = generate(model, data, processor, proxy_lang)
predictions = [clean(p) for p in predictions]
rows = []
for i, p in enumerate(predictions):
    rows.append([filepaths[i].split("/")[-1], p])
lang_df = pd.DataFrame(rows, columns=["audio_file", "sentence"])

100%|██████████| 5/5 [00:09<00:00,  1.82s/it]


In [23]:
gold_transcripts_df = pd.read_csv(
    f"/cs-data-01/drd92/mozilla-asr-challenge/mcv-sps-st-09-2025/sps-corpus-1.0-2025-09-05-{lang}/ss-corpus-{lang}.tsv",
    sep = '\t'
)

In [24]:
for i, datum in enumerate(data):
    display(Audio(datum))
    print('pred:', lang_df.loc[i]['sentence'].strip())
    print('gold:', gold_transcripts_df[gold_transcripts_df['audio_file'] == datum.split(os.path.sep)[-1]]['transcription'].item())

pred: Ya sekarang lagi musim banjir musim hujan Banjir di mana-mana Makin kita kegiatan sehari-hari makin susah ya Apalagi yang baca kolaan mah kadang pagi kita hujan
gold: iya pun sekarang lagi musim banjir musim ujan.. banjir lagi dimana-mana.. ya makin buat kita kegiatan sehari-hari makin susah yak.. apalagi yang bocah sêkolahan mah kadang pagi kita ujan..


pred: Kalau kita jalan-jalan dengan cara aman ya itu biasanya sih yang ayah alamin kalau jalan-jalan Ya kita cek-cek dulu terutama ya keuangan terus kedua ya kendaraan Kendaraan itu ya seperti yang saya gunakan itu bias di kampung motor contohnya Apa ya contohnya pemotor kita tuh banyak pempes apa kagak terus apa ada bensinnya apa kagak ya itu doang sih yang ayah alamin
gold: Kalau kita Jalan-jalan dengan cara aman ya itu biasenye si yang aye alamin kalo jalan-jalan, ya kita cek-cek dulu terutama ya keuangan, terus kedua ya kendaraan, kendaraan itu ya seperti yang saya gunakan tuh yang biasa di kampung seperti motor, apé ya contohnyé motor kite bannye kempes ape kaga, terus ape ade bensinnya ape kaga, ya itu doang si yang aye perhatiin si   


pred: Kerana kalau beli barang online itu kita tidak perlu datang ke tempat cuma scroll-scroll saja di online terus tinggal check out langsung bayar tiba-tiba nanti barangnya beberapa hari kemudian datang Tapi kita harus lihat dulu barang-barangnya itu biasanya ada bintangnya berapa Misalkan bintang 5 itu barangnya berarti sudah bagus itu tidak apa-apa untuk di check out Kenapa sering beli Jadi kita tidak perlu makan waktu untuk jalan keluar kerana kita tinggal tunggu di rumah saja
gold: karna kalo beli barang online itu kita gaperlu daténg ketempat cuman skrol skrol aja ééé di online trus di cekot langsung bayar tiba-tiba tuh barangnya beberapa hari kemudian tuh daténg tapi kita harus liat dulu barang-barangnya tuh ééé biasanya ada, ada bintangnya berapah misalkan bintang lima itu barangnya berati udah bagus nah itu gapapa di cekot gitu kenapa sering beli jadi tuh kita ga makan waktu untuk jalan keluar terutama kita tinggal tunggu dirumah aja bayar lewat hape nah itu langsung diproses

pred: Di kampung saya kalau orang hajatan pakenya kembang terus kalau orang tu hakikan tabur-taburin pake kembang kalau orang tu syukuran tu kalau bebelian apaan tu ditaburin pake kembang Saya mah begitu kampung saya ceri khas pake kembang kalau ada apa-apa
gold: lah dikampung ayé mah kalo orang, hajatan pakenya kembang, terus kalo orang tua akikahan, tabur-taburi pake kembang, kalo orang tuh sukuran tuh kalo bebelian apaan tuh ditaburi pake kembang, ayé mah begitu kampung ayé ciri khas nya pake kembang kalo ada apê-apê.


pred: Peran wadon itu di masyarakat itu banyak Dia selain ngurus di rumah tangganya sendiri dia biasanya ngurus lakinya ngurus suaminya ngurus anak-anaknya Dia juga bisa berperan di lingkungannya Biasanya itu mereka itu memiliki peran di masyarakat itu di kantor-kantor posyandu gitu kan Terus di ibu-ibu PKK Mereka itu biasanya aktif yang kayak begitu Peran perempuan di masyarakat salah satunya itu
gold: [êêê] Pêran wadon itu di masyarakat itu banyak, ya. Dia selain ngurusin rumah tangganya sendiri dia biasanya ngurus lakinya, ngurus suaminya, ngurus anak-anaknya, dia juga bisa berperan di lingkungannya [êêê] biasanya tuh ya mereka itu [êêê] memiliki peran di masyarakat itu di kantor-kantor posyandu gitukan, terus ibu-ibu PKK. Mereka tuh biasanya aktif yang kayak gitu. Peran perempuan di masyarakat salah satunya ya itu.
